<a href="https://colab.research.google.com/github/visiont3lab/machine-learning-course/blob/main/regression/Esercizio_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from sklearn.pipeline import Pipeline
import pandas as pd
from sklearn import datasets

In [4]:
def load_dataset(name="boston"):
    datasets_list = ["boston", "diabetes"] #,"tips"]
    chosen_dataset = name #"boston"

    boston = datasets.load_boston()
    df_boston = pd.DataFrame(boston.data,columns=boston.feature_names)
    df_boston["price"] = boston.target

    diabetes = datasets.load_diabetes()
    df_diabetes = pd.DataFrame(diabetes.data,columns=diabetes.feature_names)
    df_diabetes["desease"] = diabetes.target
    df_diabetes = df_diabetes.round(decimals=5)

    #df_nuovo_dataset = pd.read_csv("https://docs.google.com/spreadsheets/d/e/2PACX-1vSqHhx2kS9gCNmI04yksqTP2PRsT6ifTU2DLokKs3Y6KgcSGIAL7_4t_q_8kNhVkFA0xD2nt7hn_w-5/pub?output=csv")

    dict_datasets = {
        "boston": df_boston,
        "diabetes": df_diabetes,
    #    "nuovo" : df_nuovo_dataset
    }    
    df = dict_datasets[chosen_dataset]#.head(10)
    df = df.dropna()

    #print(df.head())
    X = df.iloc[:,0:-1].values  # Input: dalla prima alla penultima colonna
    Y = df.iloc[:,-1].values    # Target: utlima colonna

    return X,Y,df

In [12]:
X,Y,df = load_dataset(name="boston")
display(df.head())

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,price
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


1. Divido i dati in train e test
2. Preceprocessing: Lavoro con gli input. Standard Scaler, Polinomial?
3. Alleno il modello. Con o senza pipeline
4. Testo il modello (Su dati che non ho usato nel training)



## Soluzione

In [29]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_squared_error

In [10]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, shuffle=True, random_state=42)
#print(X_train.shape, X_test.shape)

(354, 13) (152, 13)


In [37]:
reg = LinearRegression(fit_intercept=True)
reg.fit(X_train,y_train)
y_hat_train = reg.predict(X_train)
mse = mean_squared_error(y_train, y_hat_train)
R2 = reg.score(X_train, y_train)
print(mse,R2)
xaxis = np.linspace(1,len(y_train),len(y_train),dtype=np.int)
#plt.plot(xaxis, y_train)
#plt.plot(xaxis,y_hat_train)

fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=y_train,mode='lines+markers', name="Y train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=y_hat_train,mode='lines+markers', name="Y Hat train")
)
fig.show()

22.545481487421423 0.7434997532004697


In [35]:
y_hat_test = reg.predict(X_test)
mse = mean_squared_error(y_test, y_hat_test)
R2 = reg.score(X_test, y_test)
print(mse,R2)
fig = go.Figure()
fig.add_trace(
    go.Scatter(x=xaxis, y=y_test,mode='lines+markers', name="Y train")
)
fig.add_trace(
    go.Scatter(x=xaxis, y=y_hat_test,mode='lines+markers', name="Y Hat train")
)
fig.show()

21.517444231176903 0.7112260057484974


In [ ]:
from sklearn.pipeline import Pipeline
from joblib import dump, load
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.model_selection import train_test_split

generate_random = np.random.RandomState(667)
x = 10 * generate_random.rand(100)
y = 3 *np.sin(x) + np.random.randn(100) # -1 0 1

X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.30, shuffle=True, random_state=42)

# creazione del modello
X_train = X_train.reshape(-1,1) # 100x1
mypipe = Pipeline([
	("std",  StandardScaler() ),  # (x-np.mean(x))/np.std(x) -----> z = (x - x_mean) / std
    ("poly",  PolynomialFeatures(degree=4) ),
	("reg", LinearRegression(fit_intercept=True))
])
mypipe.fit(X_train,y_train)
print(mypipe.named_steps["reg"].coef_,mypipe.named_steps["reg"].intercept_)
Y_hat = mypipe.predict(X_train)
dump(mypipe, 'mypipeline.joblib')

fig = plt.figure(figsize=(15,10))
plt.scatter(X_train, y_train)
plt.scatter(X_train, Y_hat)
plt.show()


In [ ]:
from joblib import dump, load
# ------------
# Produzione
loaded_pipe = load('mypipeline.joblib')
X_test = X_test.reshape(-1,1)  
Y_hat_test = loaded_pipe.predict(X_test)

fig = plt.figure(figsize=(15,10))
plt.scatter(X_test,y_test)
plt.scatter(X_test, Y_hat_test)
plt.show()